In [ ]:
import torch
from longformer.longformer import Longformer, LongformerConfig
from longformer.sliding_chunks import pad_to_window_size
from transformers import RobertaTokenizer
from transformers import AutoModel
from transformers import AutoTokenizer

config = LongformerConfig.from_pretrained(r'C:\Users\bnawa\longformer-base-4096')
config.attention_mode = 'sliding_chunks'

tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

model = AutoModel.from_pretrained("allenai/longformer-base-4096")
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
tokenizer.model_max_length = model.config.max_position_embeddings

SAMPLE_TEXT = ' '.join(['Hello world! '] * 1000)  # long input document

input_ids = torch.tensor(tokenizer.encode(SAMPLE_TEXT)).unsqueeze(0)  # batch of size 1

# TVM code doesn't work on CPU. Uncomment this if `config.attention_mode = 'tvm'`
model = model.cuda(); input_ids = input_ids.cuda()

# Attention mask values -- 0: no attention, 1: local attention, 2: global attention
attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device) # initialize to local attention
attention_mask[:, [1, 4, 21,]] =  2  # Set global attention based on the task. For example,
                                     # classification: the <s> token
                                     # QA: question tokens

# padding seqlen to the nearest multiple of 512. Needed for the 'sliding_chunks' attention
input_ids, attention_mask = pad_to_window_size(
        input_ids, attention_mask, config.attention_window[0], tokenizer.pad_token_id)

output = model(input_ids, attention_mask=attention_mask)[0]

In [16]:
output

tensor([[[-0.0487, -0.0083,  0.0357,  ..., -0.0348, -0.0800, -0.0212],
         [-0.1541,  0.2812,  0.2079,  ...,  0.3218,  0.0356,  0.0424],
         [-0.0806,  0.0276,  0.1017,  ..., -0.3952, -0.0781,  0.3135],
         ...,
         [-0.0236,  0.0741, -0.0145,  ..., -0.0990, -0.0409, -0.0745],
         [-0.0236,  0.0741, -0.0145,  ..., -0.0990, -0.0409, -0.0745],
         [-0.0236,  0.0741, -0.0145,  ..., -0.0990, -0.0409, -0.0745]]],
       grad_fn=<NativeLayerNormBackward>)